<p style="background-color:#daa"> <b>Note:</b> This notebook should be executed on system with multi-stack GPU to see intended results - <b>Intel DataCenter GPU MAX 1550</b><br><br>For example, <b>Intel DataCenter GPU MAX 1550</b> is a 2-stack GPU, and will demostrate execution difference in FLAT/COMPOSITE modes. <b>Intel DataCenter GPU MAX 1100</b> is single stack GPU, running on single stack GPU will get results but will not demonstrate any difference in FLAT/COMPOSITE mode execution</p>

# Implicit and Explicit Scaling for Multi-Stack Architecture

In this section we cover how Multi-Stack Architecture can be programmed using Implicit Scaling and Explicit Scaling

- [Multi-Stack Architecture](#Multi-Stack-Architecture)
- [Exposing the Device Hierarchy](#Exposing-the-Device-Hierarchy)
- [FLAT Mode Programming](#FLAT-Mode-Programming)
- [COMPOSITE Mode Programming](#COMPOSITE-Mode-Programming)
  - [Implicit Scaling](#Implicit-Scaling)
    - [Performance Expectations](#Performance-Expectations)
    - [Work Scheduling and Memory Distribution](#Work-Scheduling-and-Memory-Distribution)
  - [Explicit Scaling](#Explicit-Scaling)
    - [Creating Sub-Devices](#Creating-Sub-Devices)
- [Summary](#Summary)

## Multi-Stack Architecture

Intel Data Center GPU MAX series use Multi-Stack Architecture with 1 or 2 Stack. Each Stack is capable of functioning as an independent GPU entity. The Stack can execute workloads on its own.

For general applications, the multi-stack GPU is represented as a single GPU device. Applications do not care that internally GPU is constructed out of smaller Stacks, which simplifies the programming model and allows existing applications to run without any code changes. Intel GPU driver, SYCL and OpenMP parallel language runtimes work together to automatically dispatch the workloads across the stacks.

The figures below show a single Stack and 2-Stack GPU schematic:

<img src="assets/1-tile-architecture.png">

<img src="assets/2-tile-architecture.png">

Stack are connected with fast interconnect that allows efficient communication between stacks using High Bandwidth Memory (HBM).

Any Stack is capable of reading and writing to any HBM memory. For example, Stack 0 may read the local HBM memory of Stack 1. In this case, the interconnect between Stack 0 and Stack 1 is used for communication.

Stack 0 is connected to the PCI, but any Stack can read and write system memory. The same inter-Stack interconnects are used to transfer the data. Hence, Stack 0 has the shortest path to system memory among all the Stack.

Reading and writing to system memory do not require CPU involvement, GPU can perform DMA (Direct Memory Access) over PCI to system memory.

Because access to a Stack's local HBM does not involve inter-Stack interconnect, it is more efficient than cross-Stack HBM access, with lower latency and lower inter-Stack bandwidth consumption. Advanced developers can take advantage of memory locality to achieve higher performance.

## Exposing the Device Hierarchy
A multi-stack GPU card can be exposed as a single root device, or each stack can be exposed as a root device. This can be controlled via the environment variable `ZE_FLAT_DEVICE_HIERARCHY`. The allowed values for `ZE_FLAT_DEVICE_HIERARCHY` are `FLAT`, `COMPOSITE`, or `COMBINED`.

Note that, in a system with one stack per GPU card, `FLAT` and `COMPOSITE` are the same.

A multi-stack GPU card can be programmed using two modes:
- FLAT Mode Programming
- COMPOSITE Mode Programming 



## FLAT Mode Programming

In FLAT mode, each stack is exposed as a root device. The recommendation is to use FLAT mode. The FLAT mode performs well for most applications.

In FLAT mode, the driver and language runtime provide tools that expose each stack as a root device that can be programmed independently of all the other stacks.

In FLAT mode, each stack has its own memory. A kernel offloaded to a stack will run on that stack and use the memory allocated on that stack. A kernel running on a stack can access memory on other stacks in the same GPU card. However, accessing memory on a stack other than the stack it is running on will be slower.

FLAT mode is setup using environment variable as shown below:
```
ZE_FLAT_DEVICE_HIERARCHY=FLAT
```
As an example, in FLAT mode, a single GPU card with two stack GPU such as "Intel DataCenter GPU MAX 1550" will show up as 2 GPUs. Developer is responsible for writing code to make use of 2 GPUs and distribute workload, similar to multi-GPU programming.


### Build and Run in FLAT Mode
The SYCL code below shows vectoradd implementation for single device offload:

In [ ]:
!cat lab/vectoradd_single_device.cpp

Running this single device code in FLAT mode on a multi-stack GPU will offload to a single stack:

In [ ]:
!./run_vectoradd_flat_mode_no_scaling.sh

The SYCL code below shows vectoradd implementation for multi-device offload:

In [ ]:
!cat lab/vectoradd_multi_device.cpp

Running this multi-device code in FLAT mode on a multi-stack GPU will offload to all stacks in the GPU:

In [ ]:
!./run_vectoradd_flat_mode_explicit_scaling.sh

## COMPOSITE Mode Programming

In COMPOSITE mode, each GPU card is exposed as a root device. If the card contains more than one stack, then the stacks on the GPU card are exposed as subdevices.

COMPOSITE mode is setup using environment variable as shown below:
```
ZE_FLAT_DEVICE_HIERARCHY=COMPOSITE
```
As an example, in COMPOSITE mode, a single GPU card with two stack GPU such as "Intel DataCenter GPU MAX 1550" will show up as 1 GPU. Developer will write code offloading to 1 GPU, the driver and runtime will do the work of distributing workload to the 2 stacks also known as __implicit scaling__.  The Developer can also query for individual stacks in the GPU and manually write code to distribute workload to each stack also known as __explicit scaling__.

#### Implicit scaling mode
Driver and language runtimes are responsible for work distribution and multi-stack memory placement. Application sees the GPU as one monolithic device and does not care about multi-stack architecture.

#### Explicit scaling mode
User is responsible for work distribution and mutli-stack memory placement. Driver and language runtimes provide tools that expose each Stack as a separate subdevice that can be programmed independently of all the others.

### Implicit Scaling

In Implicit Scaling Mode, Driver and language runtimes are responsible for work distribution and multi-stack memory placement. Application sees the GPU as one monolithic device and does not care about multi-stack architecture.

This environment variable changes the meaning of a device to root-device. No change in application code is required. A kernel submitted to device will utilize all stacks. Similarly, memory allocation on device will span across all stacks.

A root-device is built using multiple sub-devices, also known as stacks. These stacks form a shared memory space which allows to treat a root-device as a monolithic device without the requirement of explicit communication between stacks. This section covers multi-stack programming principles using implicit scaling. When using implicit scaling, the root-device driver is responsible for distributing work to all stacks when application code launches a kernel.

#### Performance Expectations

Implicit scaling exposes resources of all stacks to a single kernel launch. For root-device with 2 stacks, a kernel has access to 2x compute peak, 2x memory bandwidth and 2x memory capacity. In the ideal case, workload performance increases by 2x. However, cache size and cache bandwidth are increased by 2x as well which can lead to better-than-linear scaling if workload fits in increased cache capacity.

Each stack is equivalent to a NUMA domain and therefore memory access pattern and memory allocation are a crucial part to achieve optimal implicit scaling performance. Workloads with a concept of locality are expected to work best with this programming model as cross-stack memory accesses are naturally minimized. Note that compute bound kernels are not impacted by NUMA domains, thus are expected to easily scale to multiple stacks with implicit scaling.

MPI applications are more efficient with implicit scaling compared to an explicit scaling approach. A single rank can utilize the entire root-device which eliminates explicit synchronization and communication between stacks. Implicit scaling automatically overlaps local memory accesses and cross-stack memory accesses in a single kernel launch.

Implicit scaling improves kernel execution time only. Serial bottlenecks will not speed up. Applications will observe no speed-up with implicit scaling if large serial bottleneck is present. Common serial bottlenecks are:

- high CPU usage
- kernel launch latency
- PCIe transfers

These will become more pronounced as kernel execution time reduces. Note that only stack-0 has PCIe connection to the host.


#### Work Scheduling and Memory Distribution

##### Memory Coloring
Any allocation in SYCL that corresponds to a shared or device allocation is colored across all stacks, meaning that allocation is divided in number-of-stacks chunks and distributed round-robin between stacks. Consider this root-device allocation:
```cpp
int *a = sycl::malloc_device<int>(N, q);
```
For a 2-stack root-device, the first half, (elements a[0] to a[N/2-1]), is physically allocated on stack-0. The remaining half, (elements a[N/2] to a[N-1]), is located on stack-1. In the future, we will introduce memory allocation APIs that allow user-defined memory coloring.

<img src="assets/2-tile-scaling.png">

__Note:__
- Memory coloring described above is applied at page size granularity. 
  - An allocation containing three pages has two pages resident on stack-0.
  - Allocations smaller or equal than page size are resident on stack-0 only.
- Using a memory pool that is based on a single allocation will break memory coloring logic. It is recommended that applications create one allocation per object to allow that object data is distributed to all stacks.


##### Static Partitioning
Scheduling of work-groups to stacks is deterministic and referred to as static partitioning. The partitioning follows a simple rule: the slowest moving dimension is divided in number-of-stacks chunks and distributed round-robin between stacks. 

Let's look at 1-dimensional kernel launch on root-device:
```cpp
q.parallel_for(N, [=](auto i) {
    //
});
```
Since there is only a single dimension it is automatically slowest dimension and partitioned between stacks by driver. For a 2-stack root-device, iterations 0 to N/2-1 are scheduled to stack-0. The remaining iterations N/2 to N-1 are executed on stack-1.

Let's look at 3-dimensional kernel launch on root-device:

```cpp
range<3> global{nz, ny, nx};
range<3> local{1, 1, 16};

cgh.parallel_for(nd_range<3>(global, local), [=](nd_item<3> item) {
    //
});
```

The slowest dimension is z and partitioned between stacks, i.e. for 2-stack root-device, all iterations from z=0 to z=nz/2-1 are executed on stack-0. The remaining iterations with z=nz/2 to z=nz-1 are scheduled to stack-1.

In case slowest moving dimension can't be divided evenly between stacks and creates an remainder imbalance larger than 5%, driver will partition next dimension if it leads to less load imbalance. This impacts kernels with odd dimensions smaller than 19 only. Examples for different kernel launches can be seen in below table (assuming local range {1,1,16}):

Work group partition to stacks:

|nz|ny|nx|Partitioned Dimension
|---|---|---|---
|512|512|512|z
|21|512|512|z
|19|512|512|y
|18|512|512|z
|19|19|512|x

In case of multi-dimensional local range in SYCL, the partitioned dimension can change. For example, for global range {38,512,512} with local range {2,1,8} driver would partition y-dimension while for local range {1,1,16} driver would partition z-dimension. OpenMP can only have a 1-dimensional local range which is created from inner most loop and thus does not impact static partitioning heuristics. OpenMP kernels created with collapse level larger than 3 correspond to 1-dimensional kernel with all for loops linearized. The linearized loop will be portioned following 1D kernel launch heuristics.

__Note:__
- Static partitioning happens at work-group granularity.
  - This implies that all work-items in a work-group are scheduled to same stack.
- A kernel with a single work-group is resident on stack-0 only.



### Explicit Scaling

In Explicit Scaling Mode, User is responsible for work distribution and mutli-stack memory placement. Driver and language runtimes provide tools that expose each stack as a separate subdevice that can be programmed independently of all the others.

#### Creating Sub-Devices

In this section we will learn how to create sub-device in SYCL that represent each stack in a multi-stack GPU device.

##### Root-device
Represents a multi-stack GPU device, containing multiple stacks.

##### Sub-Device
Represents a stack in multi-stack GPU device. The root-device in such cases can be partitioned to sub-devices, each subdevice corresponding to a physical stack.

```
vector<device> SubDevices = RootDevice.create_sub_devices<
      sycl::info::partition_property::partition_by_affinity_domain>(
      sycl::info::partition_affinity_domain::numa);
```

##### Query for individual stacks in Multi-stack device
The example below shows how to query for individual stacks (sub-devices) in Multi-stack device


In [ ]:
%%writefile lab/sub_device.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main(){
  sycl::queue q;
  sycl::device RootDevice =  q.get_device();
  std::cout << "Device: " << RootDevice.get_info<sycl::info::device::name>() << "\n";
  std::cout << "-EUs  : " << RootDevice.get_info<sycl::info::device::max_compute_units>() << "\n\n";

  //# Check if GPU can be partitioned (stacks/Stack)
  auto partitions = RootDevice.get_info<sycl::info::device::partition_max_sub_devices>();
  if(partitions > 0){
    std::cout << "-partition_max_sub_devices: " << partitions << "\n\n";
    std::vector<sycl::device> SubDevices = RootDevice.create_sub_devices<
                  sycl::info::partition_property::partition_by_affinity_domain>(
                                                  sycl::info::partition_affinity_domain::numa);
    for (auto &SubDevice : SubDevices) {
      std::cout << "Sub-Device: " << SubDevice.get_info<sycl::info::device::name>() << "\n";
      std::cout << "-EUs      : " << SubDevice.get_info<sycl::info::device::max_compute_units>() << "\n";
    }  
  } else {
    std::cout << "-cannot partition to sub-device" << "\n\n";  
  }
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
!./run_sub_device.sh

#### Context
Contexts are used for resources isolation and sharing. A SYCL context may consist of one or multiple devices. Both root-devices and sub-devices can be within single context, but they all should be of the same SYCL platform. A SYCL program created against a context with multiple devices will be built to each of the root-devices in the context. For context that consists of multiple sub-devices of the same root-device only single build (to that root-device) is needed.

#### Unified shared memory
Memory allocated against a root-device is accessible by all of its sub-devices (stacks). So if you are operating on a context with multiple sub-devices of the same root-device, then you can use malloc_device on that root-device instead of using the slower malloc_host. Remember that if using malloc_device you'd need an explicit copy out to the host if it necessary to see data there.

#### Buffer
SYCL buffers are also created against a context and are mapped to the Level-Zero USM allocation discussed above. Current mapping is as follows:

For an integrated device, the allocations are made on the host, and are accessible by the host and the device without any copying.

Memory buffers for context with sub-devices of the same root-device (possibly including the root-device itself) are allocated on that root-device. Thus they are readily accessible by all the devices in such context. The synchronization with the host is performed by SYCL RT with map/unmap doing implicit copies when necessary.

Memory buffers for context with devices from different root-devices in it are allocated on host (thus made accessible to all devices).

#### Queue
SYCL queue is always attached to a single device in a possibly multi-device context. In order of most performant to least performant, here are some typical scenarios:

##### Context associated with single sub-device
Creating a context with a single sub-device in it and the queue is attached to that sub-device (stack), in this scheme, the execution/visibility is limited to the single sub-device only, and expected to offer the best performance per stack. See a code example:
```cpp
  vector<sycl::device> SubDevices = ...;
  for (auto &D : SubDevices) {
    // Each queue is in its own context, no data sharing across them.
    auto Q = sycl::queue(D);
    Q.submit([&](sycl::handler &cgh) { ... });
  }
```

##### Context associated with multiple sub-devices
Creating a context with multiple sub-devices (multiple stacks) of the same root-device, in this scheme, queues are to be attached to the sub-devices effectively implementing "explicit scaling". In this scheme, the root-device should not be passed to such context for better performance. See a code example below:
```cpp
  vector<sycl::device> SubDevices = ...;
  auto C = sycl::context(SubDevices);
  for (auto &D : SubDevices) {
    // All queues share the same context, data can be shared across
    // queues.
    auto Q = sycl::queue(C, D);
    Q.submit([&](sycl::handler &cgh) { ... });
  }
```

##### Context associated with root device
Creating a context with a single root-device in it and the queue is attached to that root-device. In this scheme, the work will be automatically distributed across all sub-devices/stacks via "implicit scaling" by the GPU driver, which is the most simple way to enable multi-stack hardware but doesn't offer the possibility to target specific stacks. See a code example below:
```cpp
  // The queue is attached to the root-device, driver distributes to
  // sub - devices, if any.
  auto D = sycl::device(sycl::gpu_selector_v);
  auto Q = sycl::queue(D);
  Q.submit([&](sycl::handler &cgh) { ... });
```

##### Context associated with multiple root devices
Creating Contexts with multiple root-devices (multi-card). In this scheme, the most unrestrictive context with queues attached to different root-devices, which offers most sharing possibilities at the cost of slow access through host memory or explicit copies needed. See a code example:
```cpp
  auto P = sycl::platform(sycl::gpu_selector_v);
  auto RootDevices = P.get_devices();
  auto C = sycl::context(RootDevices);
  for (auto &D : RootDevices) {
    // Context has multiple root-devices, data can be shared across
    // multi - card(requires explict copying)
    auto Q = queue(C, D);
    Q.submit([&](sycl::handler &cgh) { ... });
  }
```


### Build and Run in COMPOSITE Mode
The SYCL code below shows vectoradd implementation for single device offload:

In [ ]:
!cat lab/vectoradd_single_device.cpp

Running this single device code in COMPOSITE mode on a multi-stack GPU will offload to all stacks in the GPU using implicit scaling:

In [ ]:
!./run_vectoradd_composite_mode_implicit_scaling.sh

The SYCL code below shows vectoradd implementation for sub-device offload:

In [ ]:
!cat lab/vectoradd_sub_device.cpp

Running this sub-device code in COMPOSITE mode on a multi-stack GPU will offload to all stacks in the GPU:

In [ ]:
! ./run_vectoradd_composite_mode_explicit_scaling.sh

## Summary

Below is how the different implementations of VectorAdd code will run on multi-stack GPU (Intel DataCenter GPU MAX 1550)

| FLAT Mode | COMPOSITE MODE |  
|---|---|
| Single Device VectorAdd will offload to 1 stack (other stack is unused) | Single Device VectorAdd will offload to all stacks (implicit scaling) 
| Multi-Device VectorAdd will offload to all stacks | Sub-Device VectorAdd will offload to all stacks (explicit scaling) |

The recommendation is to use FLAT mode multi-device explicit scaling, FLAT mode performs well for most applications. However, if the memory requirement is more than what is available in a single stack, then implicit scaling may be used in COMPOSITE mode, however cross stack memory access may impact performance negatively.




## Resources

- [Intel GPU Optimization Guide](https://www.intel.com/content/www/us/en/develop/documentation/oneapi-gpu-optimization-guide/top.html) - Up to date resources for Intel GPU Optimization
- [SYCL Specification](https://registry.khronos.org/SYCL/specs/sycl-2020/pdf/sycl-2020.pdf) - Latest Specification document for reference
- [SYCL Essentials Training](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/C%2B%2BSYCL/Jupyter/oneapi-essentials-training) - Learn basics of C++ SYCL Programming